In [29]:
import os
import pandas as pd
import psycopg2

# Definir os parâmetros de conexão com o PostgreSQL
usuario = 'postgres'
senha = '87a9f05e0b51d376b83d582111c280da'
host = '147.79.110.26'  # Endereço do servidor do banco
porta = '5432'           # Porta padrão do PostgreSQL
banco_de_dados = 'botai_dev'

# Definir diretórios
dir_chat_hist = 'ChatHistory'
dir_hist_by_data = 'ChatHistory/byData'

# Carrega dados

In [30]:
# Função para criar um DataFrame a partir de uma tabela do banco de dados
def carregar_tabela_postgresql(nome_tabela):
    try:
        # Estabelecendo conexão com o banco de dados
        conn = psycopg2.connect(
            dbname=banco_de_dados,
            user=usuario,
            password=senha,
            host=host,
            port=porta
        )
        
        # Query para selecionar todos os dados da tabela desejada
        query = f"SELECT * FROM {nome_tabela};"
        
        # Lendo os dados diretamente no DataFrame
        df = pd.read_sql_query(query, conn)
        
        # Fechando a conexão
        conn.close()
        
        return df
    
    except Exception as e:
        print(f"Erro ao conectar ao banco de dados: {e}")
        return None

In [31]:
# Carregar a tabelas
df_chat_history = carregar_tabela_postgresql('chat_history')
df_people = carregar_tabela_postgresql('people')
df_departments = carregar_tabela_postgresql('departments')
df_packages = carregar_tabela_postgresql('packages')
df_stages = carregar_tabela_postgresql('stages')
df_projects = carregar_tabela_postgresql('projects')
df_reports = carregar_tabela_postgresql('reports')
df_groups = carregar_tabela_postgresql('groups')

/tmp/ipykernel_1963400/3516944689.py:17: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, conn)


In [32]:
print(df_chat_history.shape)
print(df_people.shape)
print(df_departments.shape)
print(df_packages.shape)
print(df_stages.shape)
print(df_projects.shape)
print(df_reports.shape)
print(df_groups.shape)

(8299, 23)
(32, 10)
(4, 6)
(2, 6)
(5, 9)
(12, 18)
(1, 8)
(12, 9)


# Histórico de conversas
## Identifica as datas

In [33]:
# Garantir que a coluna 'created_at' esteja no formato datetime
df_chat_history['created_at'] = pd.to_datetime(df_chat_history['created_at'])

# Criar uma nova coluna 'data_formatada' com apenas o dia, mês e ano
# df_chat_history['data'] = df_chat_history['created_at'].dt.strftime('%d-%m-%Y')
df_chat_history['data'] = df_chat_history['created_at'].dt.strftime('%Y-%m-%d')

# Exibir as primeiras linhas para confirmar
df_chat_history[['created_at', 'data']].head()


,created_at,data
0,2024-05-10 13:39:00,2024-05-10
1,2024-05-10 13:39:00,2024-05-10
2,2024-05-10 13:39:00,2024-05-10
3,2024-05-10 13:40:00,2024-05-10
4,2024-05-10 13:41:00,2024-05-10


In [34]:
df_chat_history.sample()

,message_id,event,instance,receiver,groupJid,sender_id,from_me,push_name,message_type,message_content,...,longitude,sequence_number,display_name,vcard,poll_data,source,reaction_message_id,quoted_message,created_at,data
2544,9530eded-3a3b-4131-a7e3-0f4800f7808e,messages.upsert,BotAI,5522992848074,436283272096070764@g.us,5521970102409,None,Cliente 21970102409,texto,"Sim, eu vi aqui que dá pra ter pelo WhatsApp B...",...,None,None,None,None,None,None,None,None,2024-06-17 17:46:00,2024-06-17


In [35]:
df_chat_history.columns

Index(['message_id', 'event', 'instance', 'receiver', 'groupJid', 'sender_id',
       'from_me', 'push_name', 'message_type', 'message_content', 'media_url',
       'base64_data', 'reaction', 'latitude', 'longitude', 'sequence_number',
       'display_name', 'vcard', 'poll_data', 'source', 'reaction_message_id',
       'quoted_message', 'created_at', 'data'],
      dtype='object')

In [36]:
# Definir as colunas que você deseja remover
colunas_a_remover = ['instance', 'receiver', 'created_at', 'from_me']

# Remover as colunas do DataFrame
df_chat_history = df_chat_history.drop(columns=colunas_a_remover)

df_chat_history.sample()


,message_id,event,groupJid,sender_id,push_name,message_type,message_content,media_url,base64_data,reaction,latitude,longitude,sequence_number,display_name,vcard,poll_data,source,reaction_message_id,quoted_message,data
6032,b1906fb7-1788-4e97-99db-08642e4c37d0,messages.upsert,171622593129764632@g.us,5511913358299,Cliente 11913358299,texto,e no dropshipping,None,None,None,None,None,None,None,None,None,None,None,None,2024-02-01


# Identifica os grupos

In [37]:
df_chat_history.groupJid.unique()
df_chat_history.shape

(8299, 20)

In [38]:
df_groups.group_jid.unique()
df_groups.shape

(12, 9)

In [39]:
df_groups.columns

Index(['group_id', 'group_jid', 'group_name', 'package_id', 'project_id',
       'status', 'description', 'created_at', 'updated_at'],
      dtype='object')

In [40]:
# Definir as colunas que você deseja remover
colunas_a_remover = ['status', 'updated_at']

# Remover as colunas do DataFrame
df_groups = df_groups.drop(columns=colunas_a_remover)

# Renomear a coluna 'created_at' para 'grupo_created_at'
df_groups = df_groups.rename(columns={'created_at': 'grupo_created_at'})

# Exibir um sample do DataFrame atualizado
df_groups.sample()

,group_id,group_jid,group_name,package_id,project_id,description,grupo_created_at
5,7,319892153194284268@g.us,Pedro-StoreForKids,1,2,None,2024-10-04 02:25:11.187973


In [41]:
df_group_chat_hist = pd.merge(df_chat_history, df_groups, left_on='groupJid', right_on='group_jid', how='inner')

# Exibir o DataFrame unido
df_group_chat_hist.shape

(8299, 27)

In [42]:
df_group_chat_hist.sample()

,message_id,event,groupJid,sender_id,push_name,message_type,message_content,media_url,base64_data,reaction,...,reaction_message_id,quoted_message,data,group_id,group_jid,group_name,package_id,project_id,description,grupo_created_at
2408,a0c89de0-20eb-41c1-80d2-4f49c62bdc4a,messages.upsert,436283272096070764@g.us,5521970102409,Cliente 21970102409,texto,Boa tarde!,None,None,None,...,None,None,2024-06-03,9,436283272096070764@g.us,TâniaeThaissa-TajuvanHouse,1,2,None,2024-10-04 02:25:11.187973


In [43]:
# Definir as colunas que você deseja remover
colunas_a_remover = ['group_jid', 'description']

# Remover as colunas do DataFrame
df_group_chat_hist = df_group_chat_hist.drop(columns=colunas_a_remover)

# Exibir um sample do DataFrame atualizado
df_group_chat_hist.sample()

,message_id,event,groupJid,sender_id,push_name,message_type,message_content,media_url,base64_data,reaction,...,poll_data,source,reaction_message_id,quoted_message,data,group_id,group_name,package_id,project_id,grupo_created_at
7167,15132f36-f023-492e-ba37-5866e8684abe,messages.upsert,765321137722616028@g.us,555493220768,Cliente 5493220768,texto,Já aceitei,None,None,None,...,None,None,None,None,2024-07-25,10,Nitiel-Apertto,2,2,2024-10-04 02:25:11.187973


## Identifica pacotes

In [44]:
df_group_chat_hist_pck = pd.merge(df_group_chat_hist, df_packages, left_on='package_id', right_on='package_id', how='inner')

# Exibir o DataFrame unido
df_group_chat_hist_pck.shape

(8299, 30)

In [45]:
df_group_chat_hist_pck.sample()

,message_id,event,groupJid,sender_id,push_name,message_type,message_content,media_url,base64_data,reaction,...,group_id,group_name,package_id,project_id,grupo_created_at,package_name,status,description,created_at,updated_at
1964,ef55dbb9-845c-40e8-9892-b3e5e3fbe5ef,messages.upsert,176569235456585413@g.us,5511966342251,Cliente 11966342251,texto,"Boa tarde! Pessoal, desculpem, estava corrido ...",None,None,None,...,5,SamueleLeonardo-Linqtec,1,2,2024-10-04 02:25:11.187973,Pacote Especial,ativo,Pacote Especial,2024-10-03 12:57:49.363868,2024-10-03 12:57:49.363868


In [46]:
# Definir as colunas que você deseja remover
colunas_a_remover = ['status', 'created_at', 'updated_at']

# Remover as colunas do DataFrame
df_group_chat_hist_pck = df_group_chat_hist_pck.drop(columns=colunas_a_remover)
df_group_chat_hist_pck = df_group_chat_hist_pck.rename(columns={'description': 'package_description'})

# Exibir um sample do DataFrame atualizado
df_group_chat_hist_pck.sample()

,message_id,event,groupJid,sender_id,push_name,message_type,message_content,media_url,base64_data,reaction,...,reaction_message_id,quoted_message,data,group_id,group_name,package_id,project_id,grupo_created_at,package_name,package_description
8275,9cd63709-d5d1-456c-9f03-c77cd5e8eb23,messages.upsert,960751260776592837@g.us,5585911489695,Rafael Goodds,texto,Deu sim,None,None,None,...,None,None,2024-06-20,11,Tiago-AkkiShop,2,2,2024-10-04 02:25:11.187973,Pacote Global Dois Em Um,Pacote Global Dois Em Um


In [47]:
df_group_chat_hist_pck.shape

(8299, 27)

## Identifica projetos

In [48]:
df_group_chat_hist_pck_proj = pd.merge(df_group_chat_hist_pck, df_projects, left_on='project_id', right_on='project_id', how='inner')

df_group_chat_hist_pck_proj = df_group_chat_hist_pck_proj.rename(columns={'created_at': 'project_created_at','updated_at':'project_updated_at'})

# Exibir o DataFrame unido
df_group_chat_hist_pck_proj.shape

(8299, 44)

In [49]:
df_group_chat_hist_pck_proj.sample()

,message_id,event,groupJid,sender_id,push_name,message_type,message_content,media_url,base64_data,reaction,...,logo,mined_urls,contact_email,social_media_links,status,description,project_created_at,project_updated_at,estimated_deadline,closed_at
4640,0f07f5c8-f50a-40d5-ad60-72a36cd56f71,messages.upsert,024509451509359969@g.us,5558980748460,Luiza Goodds - Central de atendimento,imagem,‎IMG-20240220-WA0104.jpg (arquivo anexado),None,None,None,...,None,None,None,None,ativo,None,2024-02-19 02:23:17.387,2024-10-04 02:23:17.387113,NaT,None


# Separa por grupos

In [50]:
df_group_chat_hist = df_group_chat_hist_pck_proj.copy()

In [51]:
df_group_chat_hist.sample()

,message_id,event,groupJid,sender_id,push_name,message_type,message_content,media_url,base64_data,reaction,...,logo,mined_urls,contact_email,social_media_links,status,description,project_created_at,project_updated_at,estimated_deadline,closed_at
5236,2af9be3f-8045-4eb3-b28c-73fbabe4b4ba,messages.upsert,220191283813053475@g.us,5511960845692,Cliente 11960845692,texto,Criarei outro gmail então,None,None,None,...,None,None,None,None,ativo,None,2024-02-19 02:23:17.387,2024-10-04 02:23:17.387113,NaT,None


In [52]:
df_group_chat_hist.shape

(8299, 44)

In [53]:
df_group_chat_hist.groupJid.unique()

array(['370613401684902859@g.us', '176569235456585413@g.us',
       '436283272096070764@g.us', '025587043275878954@g.us',
       '319892153194284268@g.us', '220191283813053475@g.us',
       '024509451509359969@g.us', '171622593129764632@g.us',
       '765321137722616028@g.us', '960751260776592837@g.us'], dtype=object)

In [54]:
# Obter os valores únicos de groupJid
valores_unicos = df_group_chat_hist.groupJid.unique()

# Criar o diretório se não existir
if not os.path.exists(dir_chat_hist):
    os.makedirs(dir_chat_hist)

# Dicionário para armazenar os DataFrames separados
df_por_grupo = {}

# Loop sobre cada valor único e criar um DataFrame filtrado
for valor in valores_unicos:
    # Filtrar o DataFrame original para pegar apenas as linhas onde groupJid é igual ao valor atual
    df_filtrado = df_group_chat_hist[df_group_chat_hist['groupJid'] == valor]
    
    # Criar um nome de arquivo com o valor do groupJid
    nome_arquivo = os.path.join(dir_chat_hist, f"{valor}.csv")
    
    # Salvar o DataFrame filtrado como um arquivo CSV
    df_filtrado.to_csv(nome_arquivo, index=False)
    
    # Armazenar o DataFrame no dicionário, usando o valor de groupJid como chave
    df_por_grupo[valor] = df_filtrado
    
    print(f"Arquivo {nome_arquivo} salvo com sucesso!")

print("Todos os arquivos foram salvos com sucesso! Ótimo trabalho!")


Arquivo ChatHistory/370613401684902859@g.us.csv salvo com sucesso!
Arquivo ChatHistory/176569235456585413@g.us.csv salvo com sucesso!
Arquivo ChatHistory/436283272096070764@g.us.csv salvo com sucesso!
Arquivo ChatHistory/025587043275878954@g.us.csv salvo com sucesso!
Arquivo ChatHistory/319892153194284268@g.us.csv salvo com sucesso!
Arquivo ChatHistory/220191283813053475@g.us.csv salvo com sucesso!
Arquivo ChatHistory/024509451509359969@g.us.csv salvo com sucesso!
Arquivo ChatHistory/171622593129764632@g.us.csv salvo com sucesso!
Arquivo ChatHistory/765321137722616028@g.us.csv salvo com sucesso!
Arquivo ChatHistory/960751260776592837@g.us.csv salvo com sucesso!
Todos os arquivos foram salvos com sucesso! Ótimo trabalho!


# Separa por dia

In [55]:
df_por_grupo['370613401684902859@g.us'].sample()

,message_id,event,groupJid,sender_id,push_name,message_type,message_content,media_url,base64_data,reaction,...,logo,mined_urls,contact_email,social_media_links,status,description,project_created_at,project_updated_at,estimated_deadline,closed_at
876,18fd5537-f57f-4169-8f8e-e8e94df8c268,messages.upsert,370613401684902859@g.us,5558980748460,Luiza Goodds - Central de atendimento,texto,"Me confirma os acessos do Facebook Mari, por g...",None,None,None,...,None,None,None,None,ativo,None,2024-02-19 02:23:17.387,2024-10-04 02:23:17.387113,NaT,None


In [56]:
df_por_grupo['370613401684902859@g.us']['data'].unique()

array(['2024-05-10', '2024-05-13', '2024-05-17', '2024-07-12',
       '2024-05-14', '2024-05-16', '2024-05-20', '2024-05-21',
       '2024-05-22', '2024-05-23', '2024-05-27', '2024-05-28',
       '2024-05-31', '2024-06-04', '2024-06-06', '2024-06-07',
       '2024-06-11', '2024-06-14', '2024-06-17', '2024-06-18',
       '2024-06-21', '2024-06-25', '2024-06-26', '2024-06-27',
       '2024-06-28', '2024-07-03', '2024-07-09', '2024-07-10',
       '2024-07-11', '2024-07-24', '2024-07-17', '2024-07-22',
       '2024-07-23', '2024-07-26', '2024-07-29', '2024-07-30',
       '2024-07-31', '2024-08-02', '2024-08-05', '2024-08-06',
       '2024-08-07', '2024-08-08', '2024-08-09'], dtype=object)

In [57]:
# # Garantir que o diretório existe
# os.makedirs(dir_hist_by_data, exist_ok=True)

# # Dicionário para armazenar os DataFrames por data
# df_por_data = {}

# # Pegar os valores únicos da coluna 'data' no grupo específico
# datas_unicas = df_por_grupo['370613401684902859@g.us']['data'].unique()

# # Loop sobre as datas únicas e criar um DataFrame para cada uma
# for data in datas_unicas:
#     # Filtrar o DataFrame original para pegar apenas as linhas onde 'data' é igual à data atual
#     df_dia = df_por_grupo['370613401684902859@g.us'][df_por_grupo['370613401684902859@g.us']['data'] == data]
    
#     # Armazenar o DataFrame no dicionário, usando a data como chave
#     df_por_data[data] = df_dia
    
#     # Criar um nome de arquivo com o valor do groupJid e formatar a data
#     nome_arquivo = os.path.join(dir_hist_by_data, f"370613401684902859@g.us_{data.replace('/', '-')}.csv")
#     print(nome_arquivo)
    
#     # Salvar o DataFrame dia como um arquivo CSV
#     df_dia.to_csv(nome_arquivo, index=False)
    
#     # Exibir uma mensagem confirmando a criação do DataFrame
#     print(f"DataFrame para {data} criado com {len(df_dia)} linhas.")

# # Agora você pode acessar os DataFrames por data:
# # Exemplo: df_por_data['10/05/2024']

In [58]:
# Garantir que o diretório existe
os.makedirs(dir_hist_by_data, exist_ok=True)

# Dicionário para armazenar os DataFrames por data
df_por_data = {}

# Loop sobre cada grupo no dicionário df_por_grupo
for group_jid, df_grupo in df_por_grupo.items():
    # Pegar os valores únicos da coluna 'data' para o grupo atual
    datas_unicas = df_grupo['data'].unique()
    
    # Loop sobre as datas únicas e criar um DataFrame para cada uma
    for data in datas_unicas:
        # Filtrar o DataFrame original para pegar apenas as linhas onde 'data' é igual à data atual
        df_dia = df_grupo[df_grupo['data'] == data]
        
        # Armazenar o DataFrame no dicionário, usando o group_jid e a data como chave
        chave = f"{group_jid}_{data}"
        df_por_data[chave] = df_dia
        
        # Criar um nome de arquivo com o valor do groupJid e formatar a data
        nome_arquivo = os.path.join(dir_hist_by_data, f"{group_jid}_{data.replace('/', '-')}.csv")
        print(nome_arquivo)
        
        # Salvar o DataFrame dia como um arquivo CSV
        df_dia.to_csv(nome_arquivo, index=False)
        
        # Exibir uma mensagem confirmando a criação do DataFrame
        print(f"DataFrame para {group_jid} em {data} criado com {len(df_dia)} linhas.")

print("Todos os arquivos por data foram salvos com sucesso! Ótimo trabalho!")


ChatHistory/byData/370613401684902859@g.us_2024-05-10.csv
DataFrame para 370613401684902859@g.us em 2024-05-10 criado com 60 linhas.
ChatHistory/byData/370613401684902859@g.us_2024-05-13.csv
DataFrame para 370613401684902859@g.us em 2024-05-13 criado com 62 linhas.
ChatHistory/byData/370613401684902859@g.us_2024-05-17.csv
DataFrame para 370613401684902859@g.us em 2024-05-17 criado com 12 linhas.
ChatHistory/byData/370613401684902859@g.us_2024-07-12.csv
DataFrame para 370613401684902859@g.us em 2024-07-12 criado com 39 linhas.
ChatHistory/byData/370613401684902859@g.us_2024-05-14.csv
DataFrame para 370613401684902859@g.us em 2024-05-14 criado com 19 linhas.
ChatHistory/byData/370613401684902859@g.us_2024-05-16.csv
DataFrame para 370613401684902859@g.us em 2024-05-16 criado com 26 linhas.
ChatHistory/byData/370613401684902859@g.us_2024-05-20.csv
DataFrame para 370613401684902859@g.us em 2024-05-20 criado com 19 linhas.
ChatHistory/byData/370613401684902859@g.us_2024-05-21.csv
DataFrame p

In [59]:
df_por_data['370613401684902859@g.us_2024-05-10'].columns

Index(['message_id', 'event', 'groupJid', 'sender_id', 'push_name',
       'message_type', 'message_content', 'media_url', 'base64_data',
       'reaction', 'latitude', 'longitude', 'sequence_number', 'display_name',
       'vcard', 'poll_data', 'source', 'reaction_message_id', 'quoted_message',
       'data', 'group_id', 'group_name', 'package_id_x', 'project_id',
       'grupo_created_at', 'package_name', 'package_description',
       'package_id_y', 'stage_id', 'project_name', 'store_name', 'domain',
       'url', 'niche', 'logo', 'mined_urls', 'contact_email',
       'social_media_links', 'status', 'description', 'project_created_at',
       'project_updated_at', 'estimated_deadline', 'closed_at'],
      dtype='object')